# Loading Data and Data Processing

There are different types of data that need to be loaded and processed.
1. Texts
  * CSV files
  * Human readable textual files
2. Images
3. Audio

Data may come from different sources:
1. Locally available data
2. URL of dataset
3. Available datasets from Tensorflow and Kaggle
4. Numpy and Pandas arrays

## Texts

### CSV files

#### Loading CSV files

In the following exercise, we see how data is obtained from a CSV file whose URL is known to us and loaded to Tensorflow


In the first approach, we will directly load the data from the URL.

In [ ]:
import pandas as pd #importing the pandas library

csv_data = pd.read_csv(
    "https://storage.googleapis.com/download.tensorflow.org/data/abalone_train.csv",
    names=["Length", "Diameter", "Height", "Whole weight", "Shucked weight",
           "Viscera weight", "Shell weight", "Age"])


Print the values of the data frame

In [ ]:
csv_data

In the second approach, we will download the data to our disk and create a Pandas dataframe.

In [ ]:
from tensorflow.keras import utils # We use utils for downloading the data

url = "https://storage.googleapis.com/download.tensorflow.org/data/abalone_train.csv"

dataset_dir = utils.get_file(origin=url, cache_dir="./")


Verify whether the file is downloaded. If the above download is complete, the data will be downloaded to the **datasets** directory in the current folder.

In [ ]:
from os import listdir

files = list(listdir("./datasets"))
print(files)

In [ ]:
csv_data = pd.read_csv(
    "./datasets/abalone_train.csv",
    names=["Length", "Diameter", "Height", "Whole weight", "Shucked weight",
           "Viscera weight", "Shell weight", "Age"])

csv_data

#### Preprocessing CSV files

Suppose that our goal is to predict the age from the other features. 

We will first see the unique values of Age.

In [ ]:
print(csv_data["Age"].unique())

Now we will create separate dataframe for features.

In [ ]:
features = csv_data.copy()
labels = features.pop('Age')

In [ ]:
features

In [ ]:
labels

We see only numerical values in the features. We will now create a numpy array of these features.

In [ ]:
import numpy as np

np_features = np.array(features)

print(np_features)

**Note** It is a good practice to normalize the data before training a tensorflow model.

We will use `Normalization` from `tensorflow` for this purpose. 
Note that the function `adopt` should be run only on the test data

In [ ]:
from tensorflow.keras import layers

normalize = layers.Normalization()


normalize.adapt(np_features)

#### Using the data for training

Now we are ready to progress with the training.
We have the features and the labels ready.

In [ ]:
### Building a model
import tensorflow as tf

model = tf.keras.Sequential([
  normalize, ## Note the use of Normalization layer
  layers.Dense(64),
  layers.Dense(1)
])

model.compile(loss = tf.losses.MeanSquaredError(),
                      optimizer = tf.optimizers.Adam())


In [ ]:
## Starting the training

model.fit(np_features, labels, epochs=10)


In [ ]:
## Printing the model summary
model.summary()

### Text files

Now we will focus on the text files.

We will use Stack Overflow dataset for this part.Note that this is a zipped .tar dataset.


In [ ]:
## Download, unzip and untar the dataset

url = 'https://storage.googleapis.com/download.tensorflow.org/data/stack_overflow_16k.tar.gz'

dataset_dir = utils.get_file(
    origin=url,
    untar=True,
    cache_dir='./datasets',
    cache_subdir='')


We will now see the contents of the downloaded file(s).

In [ ]:
from os import listdir

files = list(listdir("./datasets"))
print(files)

If the download is correctly done, we can see the `train` and `test` folders.

In [ ]:
files = list(listdir("./datasets/train"))
print(files)

In [ ]:
files = list(listdir("./datasets/train/javascript"))
print(files[:5]) # print the name of first five files

In [ ]:
files = list(listdir("./datasets/test"))
print(files)

As we can see there are only two folders: `train` and `test`. It is a good practice to create datasets for training, validation and testing.

In the following code, we load the training data from the `train` folder and use 20% of the data for validation.

So far, we have loaded the complete data for training. We will now create batches of size 32.

In [ ]:
batch_size = 32
validation_size_percentage = 0.2 # To ensure 80:20 training:validation 
seed = 40 # To ensure proper shuffiling

train_batch = utils.text_dataset_from_directory(
    "./datasets/train",
    batch_size=batch_size,
    validation_split= validation_size_percentage,
    subset='training',
    seed=seed)


We will now see the details of data: text and the associated label using the first batch.

In [ ]:
for text_batch, label_batch in train_batch.take(1): # Taking into consideration the first batch
    for i in range(batch_size):
        print(f'Label: {label_batch[i].numpy()} for the text: {text_batch[i].numpy()}')


But, you may have observed that the labels are not 
'csharp', 'python', 'java', 'javascript', but rather some numbers 0,1, 2, 3.

Now, we will display the associated class names.

In [ ]:
for i, label in enumerate(train_batch.class_names):
  print("Label", i, "corresponds to", label)


Now, we will create the validation dataset. 

Recall that we used the value 'training' for the parameter subset. This time we will use the value 'validation'.

In [ ]:
validation_batch = utils.text_dataset_from_directory(
    "./datasets/train",
    batch_size=batch_size,
    validation_split= validation_size_percentage,
    subset='validation',
    seed=seed)


And finally, we will create the test batch.

In [ ]:
test_batch = utils.text_dataset_from_directory(
    "./datasets/test",
    batch_size=batch_size)


However, these datasets cannot be yet used for training, since we need vectors for working with Tensorflow. 

Our next goal is to convert the text data to associated vectors.

We will use two approaches:
1. Binary Vectorization (one-hot encoding)
2. 'int' Vectorization (integer indices for each token)

We will first start with **binary Vectorization**

In [ ]:
from tensorflow.keras.layers import TextVectorization

vocabulary_size = 10000

binary_vectorize_layer = TextVectorization(
    max_tokens=vocabulary_size,
    output_mode='binary')


In [ ]:
train_text = train_batch.map(lambda text, labels: text)
binary_vectorize_layer.adapt(train_text)


In [ ]:
for text_batch, label_batch in train_batch.take(1):
    for i in range(1):
        print("Text", text_batch[0].numpy())
        print("Label", label_batch[0].numpy())
        print("Binary Vectorization:")
        print(binary_vectorize_layer(text_batch[0])) # One-hot encoding


The above output corresponds to one-hot encoding value of the text.

We will first start with **'int' Vectorization**

Unlike 'Binary' Vectorization, we also need the maximum sequence length.

Long texts will be truncated to this maximum length.

In [ ]:
from tensorflow.keras.layers import TextVectorization

vocabulary_size = 10000
max_sequence_length = 200

int_vectorize_layer = TextVectorization(
    max_tokens=vocabulary_size,
    output_sequence_length=max_sequence_length,
    output_mode='int')


In [ ]:
train_text = train_batch.map(lambda text, labels: text)
int_vectorize_layer.adapt(train_text)


In [ ]:
for text_batch, label_batch in train_batch.take(1):
    for i in range(1):
        print("Text", text_batch[0].numpy())
        print("Label", label_batch[0].numpy())
        print("Binary Vectorization:")
        print(int_vectorize_layer(text_batch[0])) # Check the length of the sequence

To understand what the above sequence signify, we will look up in the vocabulary.

In [ ]:
for i in int_vectorize_layer(text_batch[0]):
    print(f"{i}: ", int_vectorize_layer.get_vocabulary()[i])


Now, we create functions to apply binary or 'int' vectorization to the dataset.

In [ ]:
def binary_vectorize(text, label):
    text = tf.expand_dims(text, -1)
    return binary_vectorize_layer(text), label

In [ ]:
def int_vectorize(text, label):
    text = tf.expand_dims(text, -1)
    return int_vectorize_layer(text), label

In [ ]:
binary_train_batch = train_batch.map(binary_vectorize)
binary_validation_batch = validation_batch.map(binary_vectorize)
binary_test_batch = test_batch.map(binary_vectorize)

In [ ]:
int_train_batch = train_batch.map(int_vectorize)
int_validation_batch = validation_batch.map(int_vectorize)
int_test_batch = test_batch.map(int_vectorize)

In [ ]:
# Training using binary 'Vectors'
from tensorflow.keras import losses

num_labels = 4
binary_model = tf.keras.Sequential([layers.Dense(num_labels)])

binary_model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])

history = binary_model.fit(
    binary_train_batch, validation_data=binary_validation_batch, epochs=10)


In [ ]:
print(binary_model.summary())

In [ ]:
num_labels = 4
vocab_size=vocabulary_size + 1

int_model = tf.keras.Sequential([
      layers.Embedding(vocab_size, 64, mask_zero=True),
      layers.Conv1D(64, 5, padding="valid", activation="relu", strides=2),
      layers.GlobalMaxPooling1D(),
      layers.Dense(num_labels)
  ])

int_model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])
history = int_model.fit(int_train_batch, validation_data=int_validation_batch, epochs=5)


In [ ]:
print(int_model.summary())


In [ ]:
binary_loss, binary_accuracy = binary_model.evaluate(binary_test_batch)
int_loss, int_accuracy = int_model.evaluate(int_test_batch)

print("Binary model accuracy: {:2.2%}".format(binary_accuracy))
print("Int model accuracy: {:2.2%}".format(int_accuracy))


## Images

### Loading Image dataset

In [ ]:
url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file(origin=url,
                                   cache_dir='./datasets',
                                   cache_subdir='',
                                   untar=True)


We will now check the contents of the downloded data

In [ ]:
from os import listdir

files = list(listdir("./datasets"))
print(files)

In [ ]:
files = list(listdir("./datasets/flower_photos"))
print(files)

In [ ]:
batch_size = 32
img_height = 180
img_width = 180

In [ ]:
batch_size = 32
validation_size_percentage = 0.2 # To ensure 80:20 training:validation 
seed = 40 # To ensure proper shuffiling

train_batch = utils.image_dataset_from_directory(
    "./datasets/flower_photos",
    batch_size=batch_size,
    validation_split= validation_size_percentage,
    subset='training',
    image_size=(img_height, img_width),
    seed=seed)


In [ ]:
validation_batch = utils.image_dataset_from_directory(
    "./datasets/flower_photos",
    batch_size=batch_size,
    validation_split= validation_size_percentage,
    subset='validation',
    image_size=(img_height, img_width),
    seed=seed)


In [ ]:
class_names = train_batch.class_names
print(class_names)


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_batch.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")


In [ ]:
for images, labels in train_batch.take(1):
    print(images[0].shape)
    print(labels.shape)
    break


In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1./255)

In [ ]:
normalized_batch = train_batch.map(lambda x, y: (normalization_layer(x), y))

In [ ]:
for images, labels in normalized_batch.take(1):
    print(np.min(images[0]), np.max(images[0]))


In [ ]:
num_classes = 5

model = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./255),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])

model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])


In [ ]:
model.fit(
  train_batch,
  validation_data=validation_batch,
  epochs=3
)


In [ ]:
print(model.summary())

### References
1. [Tensorflow: Load CSV data](https://www.tensorflow.org/tutorials/load_data/csv?hl=en)
2. [Tensorflow: Load text](https://www.tensorflow.org/tutorials/load_data/text?hl=en])
3. [tf.keras.utils.text_dataset_from_directory](https://www.tensorflow.org/api_docs/python/tf/keras/utils/text_dataset_from_directory)
4. [tf.keras.utils.get_file](https://www.tensorflow.org/api_docs/python/tf/keras/utils/get_file)
5. [tf.keras.layers.TextVectorization](https://www.tensorflow.org/api_docs/python/tf/keras/layers/TextVectorization)
6. [Load and preprocess images](https://www.tensorflow.org/tutorials/load_data/images?hl=en%5D)